In [ ]:
pip install gymnasium stable-baselines3 numpy shimmy

: 

In [ ]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3 import SAC

# ==========================================
# 1. 가상 스마트팜 환경 (Digital Twin) 정의
# ==========================================
class SmartFarmEnv(gym.Env):
    """
    [문서의 Digital Twin 역할]
    - 상태(State): [현재온도, 현재습도, 목표온도, 목표습도]
    - 행동(Action): [냉난방기 출력(-1~1), 가습/제습기 출력(-1~1)]
    - 보상(Reward): 목표치와의 오차가 작을수록 점수 높음 (Scalarization)
    """
    def __init__(self):
        super(SmartFarmEnv, self).__init__()

        # 행동 공간 정의:
        # Action 0: 히터/에어컨 (-1: 풀냉방, 0: 꺼짐, 1: 풀난방)
        # Action 1: 가습/제습 (-1: 풀제습, 0: 꺼짐, 1: 풀가습)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)

        # 관찰 공간 정의: [현재온도, 현재습도, 목표온도, 목표습도]
        # 온도는 0~50도, 습도는 0~100% 범위로 가정
        self.observation_space = spaces.Box(
            low=np.array([0.0, 0.0, 0.0, 0.0], dtype=np.float32),
            high=np.array([50.0, 100.0, 50.0, 100.0], dtype=np.float32),
            dtype=np.float32
        )

        # 초기 상태 설정
        self.state = None
        self.target_temp = 25.0
        self.target_hum = 60.0
        self.max_steps = 100 # 한 에피소드 길이 (모의 농사 기간)
        self.current_step = 0

        # 물리 상수
        self.ambient_temp = 20.0  # 외부 온도 (실온으로 복귀하려는 경향)
        self.ambient_hum = 50.0   # 외부 습도
        self.temp_decay = 0.05    # 실온 복귀 계수
        self.hum_decay = 0.03     # 습도 복귀 계수

    def reset(self, seed=None, options=None):
        # 환경 초기화 (농사 시작)
        super().reset(seed=seed)

        # 랜덤하게 시작 온도/습도 설정 (도메인 랜덤화 효과)
        start_temp = np.random.uniform(15.0, 35.0)
        start_hum = np.random.uniform(40.0, 80.0)

        self.state = np.array([start_temp, start_hum, self.target_temp, self.target_hum], dtype=np.float32)
        self.current_step = 0

        return self.state, {}

    def step(self, action):
        # [문서의 물리 엔진 계산 단계]
        temp_action = np.clip(action[0], -1.0, 1.0) # 냉난방 출력
        hum_action = np.clip(action[1], -1.0, 1.0)  # 가습제습 출력

        current_temp, current_hum, _, _ = self.state

        # --- 물리 법칙 시뮬레이션 (개선된 모델) ---
        # 1. 행동에 따른 변화
        dt_temp = temp_action * 1.5  # 냉난방 효과
        dt_hum = hum_action * 4.0    # 가습/제습 효과

        # 2. 자연적인 복귀력 (외부 환경으로 돌아가려는 경향)
        dt_temp += (self.ambient_temp - current_temp) * self.temp_decay
        dt_hum += (self.ambient_hum - current_hum) * self.hum_decay

        # 3. 환경 노이즈 (측정 오차 및 외란)
        noise_temp = np.random.normal(0, 0.2)
        noise_hum = np.random.normal(0, 0.8)

        # 상태 업데이트 (Next State)
        next_temp = current_temp + dt_temp + noise_temp
        next_hum = current_hum + dt_hum + noise_hum

        # 범위 제한 (Clipping)
        next_temp = np.clip(next_temp, 0.0, 50.0)
        next_hum = np.clip(next_hum, 0.0, 100.0)

        self.state = np.array([next_temp, next_hum, self.target_temp, self.target_hum], dtype=np.float32)

        # --- 보상 계산 (Reward Calculation: 정규화된 오차) ---
        # 목표와의 오차(Error) 계산 및 정규화
        temp_error = abs(next_temp - self.target_temp) / 25.0  # 온도 오차를 0~1로 정규화 (최대 ±25도 가정)
        hum_error = abs(next_hum - self.target_hum) / 50.0     # 습도 오차를 0~1로 정규화 (최대 ±50% 가정)

        # 가우시안 보상 함수 (목표에 가까울수록 높은 보상)
        temp_reward = np.exp(-temp_error**2 / 0.1)  # 온도 보상 (0~1)
        hum_reward = np.exp(-hum_error**2 / 0.1)    # 습도 보상 (0~1)

        # 최종 보상: 가중 평균 (온도를 더 중요하게)
        reward = 0.6 * temp_reward + 0.4 * hum_reward

        # 에너지 소비 페널티 (지속가능성 고려)
        energy_penalty = 0.01 * (abs(temp_action) + abs(hum_action))
        reward -= energy_penalty

        # 종료 조건
        self.current_step += 1
        terminated = False
        truncated = self.current_step >= self.max_steps

        return self.state, reward, terminated, truncated, {}

# ==========================================
# 2. SAC 에이전트 학습 (Training Loop)
# ==========================================

# 1) 환경 생성
env = SmartFarmEnv()

# 2) SAC 모델 초기화
# - Policy: "MlpPolicy" (이미지를 안 쓰므로 MLP 사용)
# - ent_coef="auto": 문서에 나온 Entropy Maximization 자동 조절
model = SAC(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=3e-4,
    buffer_size=50000,  # Replay Buffer 크기 (충분한 경험 저장)
    batch_size=256,
    learning_starts=1000,  # 학습 시작 전 수집할 경험의 양
    tau=0.005,             # Soft update 계수 (타겟 네트워크)
    gamma=0.99,            # 할인 인자 (미래 보상 중요도)
    ent_coef='auto',       # 엔트로피 자동 조정 (탐험 vs 이용 균형)
    target_update_interval=1,
    train_freq=1
)

print("--- 학습 시작 (농사 시뮬레이션 진행 중...) ---")
# 3) 학습 실행 (50,000 스텝으로 증가)
model.learn(total_timesteps=50000, log_interval=10)
print("--- 학습 완료 ---")

# ==========================================
# 3. 학습된 모델 테스트 (Inference)
# ==========================================
print("\n--- 실제 제어 테스트 ---")
obs, _ = env.reset()
total_reward = 0

for i in range(20):
    # 학습된 모델이 행동 결정 (deterministic=True는 탐험 끄고 정석대로 하라는 뜻)
    action, _state = model.predict(obs, deterministic=True)

    # 환경에 행동 적용
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward

    current_temp = obs[0]
    current_hum = obs[1]
    target_temp = obs[2]
    target_hum = obs[3]

    print(f"Step {i+1}: 온도={current_temp:.2f}°C (목표: {target_temp:.1f}°C), "
          f"습도={current_hum:.2f}% (목표: {target_hum:.1f}%) | "
          f"행동: [난방 {action[0]:+.2f}, 습도 {action[1]:+.2f}] | 보상: {reward:.3f}")

    if terminated or truncated:
        break

print(f"\n평균 보상: {total_reward / (i+1):.3f}")

# 모델 저장
model.save("sac_smartfarm_agent")
print("\n모델이 'sac_smartfarm_agent.zip'로 저장되었습니다.")